In [1]:
%load_ext sparksql_magic

from pyspark.sql import SparkSession

# Start Spark session
spark = (
    SparkSession.builder.appName("FilterGHJson")
    .config("spark.sql.catalogImplementation", "hive")
    .enableHiveSupport()
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/02/21 12:31:08 WARN Utils: Your hostname, NathanBlade15 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/02/21 12:31:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/21 12:31:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/21 12:31:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.json("../data/*.json.gz")

25/02/21 12:31:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
df.printSchema()
df.show()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|             type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|{https://avatars....|2025-01-01T01:00:00Z|45186095831|                NULL|{created, NULL, {...|  true|{810155444, aws-a...|IssueCommentEvent|
|{https://avatars....|2025-01-01T01:00:00Z|45186095832|                NULL|{NULL, 1c0bb7ae07...|  true|{314538798, Malee...|        PushEvent|
|{https://avatars....|2025-01-01T01:00:00Z|45186095834|                NULL|{NULL, c2e22ed422...|  true|{463061974, phanu...|        PushEvent|
|{https://avatars....|2025-01-01T01:00:00Z|45186095837|                NULL|{NULL, 40f81c7a80...|  true|{894756355, plume...|        Pus

In [4]:
df = df.filter(df["type"] == "IssuesEvent")

In [5]:
df.createOrReplaceTempView("df")

In [6]:
df.show(20)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|       type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------+
|{https://avatars....|2025-01-01T01:00:02Z|45186096107|                NULL|{opened, NULL, NU...|  true|{645980442, mo9a7...|IssuesEvent|
|{https://avatars....|2025-01-01T01:00:02Z|45186096173|                NULL|{opened, NULL, NU...|  true|{606520610, Purfv...|IssuesEvent|
|{https://avatars....|2025-01-01T01:00:04Z|45186096455|                NULL|{opened, NULL, NU...|  true|{903529255, carol...|IssuesEvent|
|{https://avatars....|2025-01-01T01:00:04Z|45186096537|                NULL|{closed, NULL, NU...|  true|{439100086, Price...|IssuesEvent|
|{https://avatars....|2025-01-01T0

In [15]:
spark.sql("""
SELECT 
    CAST(payload.issue.closed_at AS TIMESTAMP) AS closed_at,
    CAST(payload.issue.created_at AS TIMESTAMP) AS created_at,
    (unix_timestamp(CAST(payload.issue.closed_at AS TIMESTAMP)) - unix_timestamp(CAST(payload.issue.created_at AS TIMESTAMP))) / 3600 AS duration_hours,
    payload.action, 
    payload.issue.*
FROM df
WHERE payload.action = 'closed' 
    AND payload.issue.closed_at IS NOT NULL
""").show(10)


+-------------------+-------------------+--------------------+------+------------------+--------+---------+------------------+-----------------------------+--------------------+--------+--------------------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------------+------+---------+------------------+------+------------------------+------------+--------------------+--------------------+------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          closed_at|         created_at|      duration_hours|action|active_lock_reason|assignee|assignees|author_association|                         body|           closed_at|comments|        comments_url|          created_at|draft|          events_url|            html_url|        id|              labels|          labels_url|locked|milestone|           node_id|number|performed_via_github_app|pull_r

In [16]:
spark.sql("""
SELECT 
    CAST(payload.issue.closed_at AS TIMESTAMP) AS closed_at,
    CAST(payload.issue.created_at AS TIMESTAMP) AS created_at,
    (unix_timestamp(CAST(payload.issue.closed_at AS TIMESTAMP)) - unix_timestamp(CAST(payload.issue.created_at AS TIMESTAMP))) / 3600 AS duration_hours,
    payload.action, 
    payload.issue.*
FROM df
""").show(10)

+-------------------+-------------------+-----------------+------+------------------+--------+---------+------------------+-----------------------------+--------------------+--------+--------------------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------------+------+---------+------------------+------+------------------------+------------+--------------------+--------------------+------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          closed_at|         created_at|   duration_hours|action|active_lock_reason|assignee|assignees|author_association|                         body|           closed_at|comments|        comments_url|          created_at|draft|          events_url|            html_url|        id|              labels|          labels_url|locked|milestone|           node_id|number|performed_via_github_app|pull_request

In [21]:
%%sparksql
SELECT 
    payload.issue.id AS issue_id,
    ARRAY_SORT(
        COLLECT_LIST(NAMED_STRUCT('action', payload.action, 'timestamp', created_at)),
        (x, y) -> CASE WHEN x.timestamp < y.timestamp THEN -1 ELSE 1 END
    ) AS event_history
FROM df
GROUP BY payload.issue.id
HAVING SIZE(COLLECT_LIST(payload.action)) = 2;





only showing top 20 row(s)


issue_id,event_history
175296384,"[Row(action='closed', timestamp='2025-01-01T05:06:52Z'), Row(action='closed', timestamp='2025-01-01T05:06:52Z')]"
261975208,"[Row(action='closed', timestamp='2025-01-01T17:14:39Z'), Row(action='reopened', timestamp='2025-01-01T17:16:05Z')]"
360646678,"[Row(action='closed', timestamp='2025-01-01T19:09:21Z'), Row(action='closed', timestamp='2025-01-01T19:09:21Z')]"
421403206,"[Row(action='reopened', timestamp='2025-01-01T16:07:20Z'), Row(action='closed', timestamp='2025-01-01T23:42:35Z')]"
424486938,"[Row(action='closed', timestamp='2025-01-01T07:55:23Z'), Row(action='closed', timestamp='2025-01-01T07:56:46Z')]"
465859330,"[Row(action='closed', timestamp='2025-01-01T18:47:32Z'), Row(action='closed', timestamp='2025-01-01T18:47:35Z')]"
545338804,"[Row(action='closed', timestamp='2025-01-01T19:24:34Z'), Row(action='closed', timestamp='2025-01-01T19:24:34Z')]"
566441372,"[Row(action='closed', timestamp='2025-01-01T02:55:15Z'), Row(action='closed', timestamp='2025-01-01T02:55:15Z')]"
683591910,"[Row(action='closed', timestamp='2025-01-01T17:59:47Z'), Row(action='reopened', timestamp='2025-01-01T17:59:52Z')]"
810038935,"[Row(action='closed', timestamp='2025-01-01T14:50:08Z'), Row(action='reopened', timestamp='2025-01-01T14:50:20Z')]"


In [ ]:
spark.sql(
    """
          SELECT payload.* FROM df
          where type = 'IssueCommentEvent' or type = 'IssuesEvent'
          """
).show(20)

In [ ]:
spark.sql(
    """
          SELECT payload.issue.* FROM df
          where type = 'IssueCommentEvent'
          """
).show(20, True)

In [ ]:
spark.sql(
    """
          SELECT payload.issue.user.* FROM df
          where type = 'IssueCommentEvent'
          """
).show(20, True)

In [ ]:
%%sparksql
SELECT 
    created_at, 
    payload.issue.author_association, 
    payload.action, 
    payload.issue.closed_at, 
    payload.issue.title, 
    payload.issue.id, 
    payload.issue.labels, 
    payload.issue.state, 
    payload.issue.state_reason
FROM df 
WHERE (payload.action = 'opened' OR payload.action = 'reopened') 
    AND type = 'IssuesEvent' AND payload.issue.closed_at IS NOT NULL

In [ ]:
%%sparksql
SELECT created_at, type, payload.action, payload.issue.state, payload.issue.closed_at 
FROM df 
WHERE payload.issue.id = 2764760625 
ORDER BY created_at ASC;


In [ ]:
%%sparksql
SELECT 
    payload.issue.id AS issue_id, 
    MIN(created_at) AS first_opened, 
    MAX(payload.issue.closed_at) AS final_closed,
    TIMESTAMPDIFF(SECOND, MIN(created_at), MAX(payload.issue.closed_at)) AS resolution_time_seconds
FROM df
WHERE type = 'IssuesEvent' 
AND payload.issue.state = 'closed' 
AND payload.action = 'opened'
GROUP BY payload.issue.id
ORDER BY resolution_time_seconds DESC;


In [ ]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        created_at AS opened_at
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        created_at AS closed_at
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    TIMESTAMPDIFF(SECOND, o.opened_at, c.closed_at) AS time_to_close_seconds
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
ORDER BY time_to_close_seconds ASC;


In [ ]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MIN(created_at) AS opened_at  -- Pick first open event per issue
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')
    GROUP BY payload.issue.id
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MAX(created_at) AS closed_at  -- Pick last closed event per issue
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'
    GROUP BY payload.issue.id
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    TIMESTAMPDIFF(SECOND, o.opened_at, c.closed_at) AS time_to_close_seconds
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
WHERE c.closed_at >= o.opened_at  -- Ensure closed_at is after opened_at
ORDER BY time_to_close_seconds ASC;


In [ ]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MIN(created_at) AS opened_at,  -- Pick first open event per issue
        FIRST(payload.issue.author_association) AS author_association,  
        FIRST(payload.issue.title) AS title,
        FIRST(payload.issue.labels) AS labels,
        FIRST(payload.issue.state) AS state,
        FIRST(payload.issue.state_reason) AS state_reason,
        FIRST(payload.issue.body) AS body
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')
    GROUP BY payload.issue.id
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MAX(created_at) AS closed_at  -- Pick last closed event per issue
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'
    GROUP BY payload.issue.id
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    TIMESTAMPDIFF(SECOND, o.opened_at, c.closed_at) AS time_to_close_seconds,
    o.author_association,
    o.title,
    o.state,
    o.state_reason,
    o.body
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
WHERE c.closed_at >= o.opened_at AND o.state != 'open' -- Ensure closed_at is after opened_at
ORDER BY time_to_close_seconds DESC;


In [ ]:
%% sparksql
select * from df where type = 'IssuesEvent' and payload.action = 'opened' or payload.action = 'reopened' order by created_at desc limit 10;